## 学习尝试用theano搭建多层GRU网络

In [1]:
import os
import re
import random
import itertools
import nltk
import operator
import numpy as np
import theano
import theano.tensor as T
from numpy.random import uniform as uniform
from numpy import sqrt as sqrt

Using cuDNN version 7005 on context None
Mapped name None to device cuda: GeForce GTX 1050 Ti (0000:01:00.0)


In [2]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='utf8') as f:
        for line in f:
            #if (line % 100 == 0):
            #print(line)
            values = line.split(" +++$+++ ")
           # print(values)
            # Extract fields
            lineObj = {}
            #print(fields)
            for i, field in enumerate(fields):
                #if (i % 100 == 0):
                #print(i,field,values[i])
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            #print(values)
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            utterance_id_pattern = re.compile('L[0-9]+')
            lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
#trimmed  是否修剪
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD
#
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold过滤低频词
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)
MAX_LENGTH = 15  # Maximum sentence length to consider需要考虑的最大句子长度
datafile = os.path.join( "data/训练数据.txt")


# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    #lower()到小写
    #Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
    s = s.lower().strip()
    #与大多数编程语言相同，知正则表达式里使用"\"作为转义字符，这就可能造成反斜杠困扰。假如你需要匹配文本中的字符"\"，
    #那么使用编程道语言表示的正则表达式版里将需要4个反斜杠"\\\\"：前两个和后两个分别用于在编程语言里转义成反斜杠，
    #转换成两个反斜杠后再在正则表达式里转义成一个反斜杠。Python里的原生字符串很好地解决了这个问题，
    #这个例子中的正则表达式可以使用r"\\"表示。同样，匹配一个数字的"\\d"可以写成r"\d"。有了原生字符串，
    #你再也不用担心是不是漏写了反斜杠，写出来的表达式也更直观。
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[,']+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object读取查询/响应对并返回voc对象
def readVocs(datafile, corpus_name):
    print("Reading lines..."+corpus_name)
    print("datafile..."+datafile)
    # Read the file and split into lines

    lines = open(datafile, encoding='utf-8').\
    read().strip().split('\n')#这里反斜杆是续行符
    # Split every line into pairs and normalize 将每一行分成几对并归一化
    pairs = [[normalizeString(s) for s in l.split('【*分割*】')] for l in lines]
    #实例化类Voc
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
#如果对“ p”中的两个句子都在MAX_LENGTH阈值以下，则返回True
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token输入序列需要保留EOS令牌的最后一个字
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition使用filterPair条件的过滤器对
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
# 使用上面定义的函数，返回填充的voc对象和配对列表
def loadPrepareData(corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)#这里读取会话对
    print("Read {!s} sentence pairs".format(len(pairs)))#格式化字符串操作
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs加载/组装voc和对
save_dir = os.path.join("data", "save")
print("save_dir",save_dir)
#os.path.join 运行路径中加入新的文件夹

voc, pairs = loadPrepareData(corpus_name, datafile, save_dir)
# Print some pairs to validate打印一些对以进行验证
print("\npairs:")
for pair in pairs[:2]:
    print(pair)#pairs数组 成员是字符串数组


save_dir data\save
Start preparing training data ...
Reading lines...cornell movie-dialogs corpus
datafile...data/训练数据.txt
Read 529586 sentence pairs
Trimmed to 114074 sentence pairs
Counting words...
Counted words: 86302

pairs:
['这场 也 有 免费 的 直播 ？ 达胜', '达胜 24 直播 网有 免费 高清 的']
['这种 饼 也 只有 奉先能 吃 不然 又 闷平 交出 签名 车牌号', '交出 签名 车牌号 juc - 635']


In [3]:
MIN_COUNT = 10    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc语音中MIN_COUNT下使用的修饰词
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words过滤掉带有修饰词的词对
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence检查输入句子
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)
SENTENCE_START_TOKEN = "SENTENCE_START"
SENTENCE_END_TOKEN = "SENTENCE_END"
UNKNOWN_TOKEN = "UNKNOWN_TOKEN"
b=nltk.word_tokenize(pairs[0][0])

会话对_分割1= [nltk.word_tokenize('SENTENCE_START '+pair[0]+' SENTENCE_END')+nltk.word_tokenize('SENTENCE_START '+pair[1]+' SENTENCE_END') for pair in pairs]
会话对_分割= [ [nltk.word_tokenize('SENTENCE_START '+对+' SENTENCE_END')for 对 in pair]for pair in pairs]
词频 = nltk.FreqDist(itertools.chain(*会话对_分割1))
词汇数量=len(词频.items())

词汇_降序 = sorted(词频.items(), key=lambda x: (x[1], x[0]), reverse=True)[:词汇数量-2]
词汇_升序 = sorted(词汇_降序, key=operator.itemgetter(1))
下标_到_单词 = ["<MASK/>", UNKNOWN_TOKEN] + [x[0] for x in 词汇_升序]
单词_到_下标 = dict([(w, i) for i, w in enumerate(下标_到_单词)])
for i, 分割 in enumerate(会话对_分割):
    会话对_分割[i] =[[w if w in 单词_到_下标 else UNKNOWN_TOKEN for w in 部]for 部 in 分割]
X_train = np.asarray([[单词_到_下标[w] for w in 分割[0][:-1]] for 分割 in 会话对_分割])
y_train = np.asarray([[单词_到_下标[w] for w in 分割[1][1:]] for 分割 in 会话对_分割])

keep_words 13451 / 86299 = 0.1559
Trimmed from 114074 pairs to 40345, 0.3537 of total


In [4]:
def 打包训练数据(输入,输出,包数=10,包大小=10):
    包_输入_集=[]
    输入_遮罩_集=[]
    for 数1 in range(包数):
        包_输入=[]
        P=[i for i in range(数1*包大小,数1*包大小+(包大小))]

        NP=输入[P]
        记录长度=0
        for j in NP:
            if len(j)>记录长度:
                记录长度=len(j)
        计数=0
        for j in NP:
            补齐数=记录长度-len(j)
            输入_遮罩_临=np.ones(len(j))
            for k in range(补齐数):
                j.append(0)
                输入_遮罩_临=np.append(输入_遮罩_临,0.)
            #包_输入.append(j)
        #print(len(包_输入))
            if 计数==0:
                包_输入=np.array(j).reshape(记录长度,1)
                输入_遮罩=输入_遮罩_临.reshape(记录长度,1)
            else:
                输入_遮罩=np.concatenate((输入_遮罩,输入_遮罩_临.reshape(记录长度,1)),axis=1)
                包_输入=np.concatenate((包_输入,np.array(j).reshape(记录长度,1)),axis=1)
            计数+=1
        包_输入_集.append(包_输入) 
        输入_遮罩_集.append(输入_遮罩)
        
    输出_遮罩_集=[]
    包_输出_集=[]
    for 数1 in range(包数):
        包_输出=[]
        P=[i for i in range(数1*包大小,数1*包大小+(包大小))]

        NP=输出[P]
        记录长度=0
        for j in NP:
            if len(j)>记录长度:
                记录长度=len(j)
        计数=0
        for j in NP:
            补齐数=记录长度-len(j)
            输出_遮罩_临=np.ones(len(j))
            for k in range(补齐数):
                j.append(0)
                输出_遮罩_临=np.append(输出_遮罩_临,0.)
            if 计数==0:
                包_输出=np.array(j).reshape(记录长度,1)
                输出_遮罩=输出_遮罩_临.reshape(记录长度,1)
            else:
                
                输出_遮罩=np.concatenate((输出_遮罩,输出_遮罩_临.reshape(记录长度,1)),axis=1)
                包_输出=np.concatenate((包_输出,np.array(j).reshape(记录长度,1)),axis=1)
            计数+=1
        包_输出_集.append(包_输出)
        输出_遮罩_集.append(输出_遮罩)
    return 包_输入_集 , 包_输出_集 ,输入_遮罩_集,输出_遮罩_集

包_输入 , 包_输出,输入_遮罩_集,输出_遮罩_集=打包训练数据(X_train,y_train,200,200)

In [5]:
class 预处理_正交():
	def __init__(self, 词向量尺寸=100, 隐藏层尺寸=300):
		self.隐藏层尺寸 = 隐藏层尺寸

		E = uniform(-sqrt(1. / 词向量尺寸), sqrt(1. / 词向量尺寸), (隐藏层尺寸, 词向量尺寸))
		self.E = theano.shared(name='E', value=E.astype(theano.config.floatX))
		self.mE = theano.shared(name='mE', value=np.zeros(E.shape).astype(theano.config.floatX))
		self.参数 = [self.E]
		self.rms = [self.mE]
	def 前向传播(self,输入):

		def 前向_单步(输入_E,E_1):
			输出_E = E_1[:,输入_E]
			return 输出_E

		单层输出, updates = theano.scan(
			fn = 前向_单步,
			sequences = [ 输入],
			non_sequences = [self.E]
			)
		return 单层输出

In [6]:
class GRU_单位_集束():
	def __init__(self,  隐藏层尺寸=300, 求导深度=-1):
		self.隐藏层尺寸 = 隐藏层尺寸
		self.求导深度 = 求导深度

		U = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))
		W = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))

		self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
		self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
		self.mU = theano.shared(name='mU', value=np.zeros(U.shape).astype(theano.config.floatX))
		self.mW = theano.shared(name='mW', value=np.zeros(W.shape).astype(theano.config.floatX))
		self.参数 = [self.W, self.U]
		self.rms= [self.mW, self.mU]
	def 前向传播(self,输入,遮罩,集束量,隐层_单步_0=None):
		if 隐层_单步_0 == None:
			隐层_单步_0=T.zeros((self.隐藏层尺寸,集束量))
#			隐层_单步_0 = T.alloc(np.asarray(0., dtype=theano.config.floatX), self.隐藏层尺寸, 集束量)
		def 前向_单步( 输入_E, 遮罩,隐层_单步_前,W,U,隐藏层尺寸):
			更新门 = T.nnet.hard_sigmoid(U[0:隐藏层尺寸, :].dot(输入_E) + W[0:隐藏层尺寸, :].dot(隐层_单步_前))
			重置门 = T.nnet.hard_sigmoid(
				U[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(输入_E) + W[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(
					隐层_单步_前))
			记忆单元 = T.tanh(
				U[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(输入_E) + W[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(
					隐层_单步_前 * 重置门))
			隐层_单步 = (T.ones_like(更新门) - 更新门) * 记忆单元 + 更新门 * 隐层_单步_前
			隐层_单步 = 遮罩[None,:]*隐层_单步 + (1.0 - 遮罩)[None,:]*隐层_单步_前            
			return 隐层_单步
		单层输出, updates = theano.scan(
			fn = 前向_单步,
			truncate_gradient=self.求导深度,
			sequences = [ 输入,遮罩],
			outputs_info = [隐层_单步_0],
			non_sequences = [self.W, self.U,self.隐藏层尺寸]
			)
		return 单层输出

In [7]:
class GRU_注意力():
	def __init__(self,  隐藏层尺寸=300, 注意力尺寸=300,求导深度=-1):
		self.隐藏层尺寸 = 隐藏层尺寸
		self.求导深度 = 求导深度

		W = np.random.uniform(-np.sqrt(1. / np.sqrt(注意力尺寸*隐藏层尺寸)), np.sqrt(1. / np.sqrt(注意力尺寸*隐藏层尺寸)), (注意力尺寸, 隐藏层尺寸))
		V = np.random.uniform(-np.sqrt(1. / np.sqrt(注意力尺寸)), np.sqrt(1. / np.sqrt(注意力尺寸)), (注意力尺寸))

		self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
		self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
		self.mW = theano.shared(name='mW', value=np.zeros(W.shape).astype(theano.config.floatX))
		self.mV = theano.shared(name='mV', value=np.zeros(V.shape).astype(theano.config.floatX))
		self.参数 = [self.W, self.V]
		self.rms= [self.mW, self.mV]
	def 前向传播(self,输入):
		def 前向_单步_1( 输入_E,W,V):

			E = V.dot(T.tanh(W.dot(输入_E)))

			return E
		E, updates = theano.scan(
			fn = 前向_单步_1,
			truncate_gradient=self.求导深度,
			sequences = [ 输入],
			non_sequences = [self.W, self.V]
			)
		E_sum=T.sum(E,axis=0)
		def 前向_单步_2( E,E_sum):
           
			注意力_分=E/E_sum

			return 注意力_分
		注意力分配, updates = theano.scan(
			fn = 前向_单步_2,
			truncate_gradient=self.求导深度,
			sequences = [ E],
			non_sequences = [E_sum]
			)
		def 前向_单步_3( 输入_E,注意力_分):
			AH_分=注意力_分*输入_E 
             
			return AH_分                      
		AH_分_集, updates = theano.scan(
			fn = 前向_单步_3,
			truncate_gradient=self.求导深度,
			sequences = [输入, 注意力分配]
			)
#		AH_分_集 = theano.printing.Print('this is AH_分_集 value')(AH_分_集)
		隐层_新=T.sum(AH_分_集,axis=0)
		return 隐层_新

In [8]:
class GRU_单位_集束_A():
	def __init__(self,  隐藏层尺寸=300, 求导深度=-1):
		self.隐藏层尺寸 = 隐藏层尺寸
		self.求导深度 = 求导深度

		U = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))
		W = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))

		self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
		self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
		self.mU = theano.shared(name='mU', value=np.zeros(U.shape).astype(theano.config.floatX))
		self.mW = theano.shared(name='mW', value=np.zeros(W.shape).astype(theano.config.floatX))
		self.参数 = [self.W, self.U]
		self.rms= [self.mW, self.mU]
	def 前向传播(self,输入,遮罩,集束量,隐层_单步_0=None):
		if 隐层_单步_0 == None:
			隐层_单步_0=T.zeros((self.隐藏层尺寸,集束量))
#			隐层_单步_0 = T.alloc(np.asarray(0., dtype=theano.config.floatX), self.隐藏层尺寸, 集束量)
		def 前向_单步( 输入_E, 遮罩,隐层_单步_前,W,U,隐藏层尺寸):
			更新门 = T.nnet.hard_sigmoid(U[0:隐藏层尺寸, :].dot(输入_E) + W[0:隐藏层尺寸, :].dot(隐层_单步_前))
			重置门 = T.nnet.hard_sigmoid(
				U[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(输入_E) + W[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(
					隐层_单步_前))
			记忆单元 = T.tanh(
				U[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(输入_E) + W[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(
					隐层_单步_前 * 重置门))
			隐层_单步 = (T.ones_like(更新门) - 更新门) * 记忆单元 + 更新门 * 隐层_单步_前
#			隐层_单步 = 遮罩[None,:]*隐层_单步 + (1.0 - 遮罩)[None,:]*隐层_单步_前            
			return 隐层_单步
		单层输出, updates = theano.scan(
			fn = 前向_单步,
			truncate_gradient=self.求导深度,
			sequences = [ 输入,遮罩],
			outputs_info = [隐层_单步_0],
			non_sequences = [self.W, self.U,self.隐藏层尺寸]
			)
		return 单层输出

In [9]:
class GRU_单位2_集束():
	def __init__(self,  隐藏层尺寸=300, 求导深度=-1):
		self.隐藏层尺寸 = 隐藏层尺寸
		self.求导深度 = 求导深度

		U = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))
		W = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))

		self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
		self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
		self.mU = theano.shared(name='mU', value=np.zeros(U.shape).astype(theano.config.floatX))
		self.mW = theano.shared(name='mW', value=np.zeros(W.shape).astype(theano.config.floatX))
		self.参数 = [self.W, self.U]
		self.rms= [self.mW, self.mU]
	def 前向传播(self,遮罩,集束量,隐层_单步_0=None):
		if 隐层_单步_0 == None:
			隐层_单步_0=T.zeros((self.隐藏层尺寸,集束量))
#			隐层_单步_0 = T.alloc(np.asarray(0., dtype=theano.config.floatX), self.隐藏层尺寸, 集束量)
		def 前向_单步( 遮罩,隐层_单步_前,W,U,隐藏层尺寸):
			更新门 = T.nnet.hard_sigmoid(U[0:隐藏层尺寸, :].dot(隐层_单步_前) + W[0:隐藏层尺寸, :].dot(隐层_单步_前))
			重置门 = T.nnet.hard_sigmoid(
				U[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(隐层_单步_前) + W[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(
					隐层_单步_前))
			记忆单元 = T.tanh(
				U[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(隐层_单步_前) + W[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(
					隐层_单步_前 * 重置门))
			隐层_单步 = (T.ones_like(更新门) - 更新门) * 记忆单元 + 更新门 * 隐层_单步_前
			隐层_单步 = 遮罩[None,:]*隐层_单步 + (1.0 - 遮罩)[None,:]*隐层_单步_前 
			return 隐层_单步
		单层输出, updates = theano.scan(
			fn = 前向_单步,
			sequences = [ 遮罩],
			truncate_gradient=self.求导深度,
			outputs_info = [隐层_单步_0],
			non_sequences = [self.W, self.U,self.隐藏层尺寸])
		return 单层输出

In [10]:
class GRU_单位2_集束_A():
	def __init__(self,  隐藏层尺寸=300, 求导深度=-1):
		self.隐藏层尺寸 = 隐藏层尺寸
		self.求导深度 = 求导深度

		U = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))
		W = np.random.uniform(-np.sqrt(1. / 隐藏层尺寸), np.sqrt(1. / 隐藏层尺寸), (3 * 隐藏层尺寸, 隐藏层尺寸))

		self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
		self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
		self.mU = theano.shared(name='mU', value=np.zeros(U.shape).astype(theano.config.floatX))
		self.mW = theano.shared(name='mW', value=np.zeros(W.shape).astype(theano.config.floatX))
		self.参数 = [self.W, self.U]
		self.rms= [self.mW, self.mU]
	def 前向传播(self,遮罩,集束量,隐层_单步_0=None):
		if 隐层_单步_0 == None:
			隐层_单步_0=T.zeros((self.隐藏层尺寸,集束量))
#			隐层_单步_0 = T.alloc(np.asarray(0., dtype=theano.config.floatX), self.隐藏层尺寸, 集束量)
		def 前向_单步( 遮罩,隐层_单步_前,W,U,隐藏层尺寸):
			更新门 = T.nnet.hard_sigmoid(U[0:隐藏层尺寸, :].dot(隐层_单步_前) + W[0:隐藏层尺寸, :].dot(隐层_单步_前))
			重置门 = T.nnet.hard_sigmoid(
				U[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(隐层_单步_前) + W[1 * 隐藏层尺寸:2 * 隐藏层尺寸, :].dot(
					隐层_单步_前))
			记忆单元 = T.tanh(
				U[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(隐层_单步_前) + W[2 * 隐藏层尺寸:3 * 隐藏层尺寸, :].dot(
					隐层_单步_前 * 重置门))
			隐层_单步 = (T.ones_like(更新门) - 更新门) * 记忆单元 + 更新门 * 隐层_单步_前
#			隐层_单步 = 遮罩[None,:]*隐层_单步 + (1.0 - 遮罩)[None,:]*隐层_单步_前 
			return 隐层_单步
		单层输出, updates = theano.scan(
			fn = 前向_单步,
			sequences = [ 遮罩],
			truncate_gradient=self.求导深度,
			outputs_info = [隐层_单步_0],
			non_sequences = [self.W, self.U,self.隐藏层尺寸])
		return 单层输出

In [11]:
class 后处理_输出():
	def __init__(self, 词向量尺寸=100, 隐藏层尺寸=300):
		self.隐藏层尺寸 = 隐藏层尺寸

		V = uniform(-sqrt(1./隐藏层尺寸), sqrt(1./隐藏层尺寸), (词向量尺寸, 隐藏层尺寸))
		self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
		self.mV = theano.shared(name='mV', value=np.zeros(V.shape).astype(theano.config.floatX))
		self.参数 = [self.V]
		self.rms = [self.mV]
	def 前向传播(self,输入):

		def 前向_单步(输入_V,V_1):
			输出_V = V_1.dot(输入_V)
			return 输出_V

		单层输出, updates = theano.scan(
			fn = 前向_单步,
			sequences = [ 输入],
			non_sequences = [self.V]
			)
		return 单层输出

In [12]:
class GRU_集束():
	def __init__(self, 词向量尺寸=100, 隐藏层尺寸=300,层数=2,求导深度=-1):
		self.层数=层数
		self.词向量尺寸=词向量尺寸
		self.隐藏层尺寸=隐藏层尺寸
		self.求导深度=求导深度
		self.参数=[]
		self.rms=[]
		输入 = T.imatrix('i')
		目标输出  = T.imatrix('o')
		输入_遮罩= T.fmatrix('mi')
		输出_遮罩= T.fmatrix('m0')        
		目标长度 = T.iscalar("k") 
		集束量= T.iscalar("b") 
        
		预处理=预处理_正交(self.词向量尺寸,self.隐藏层尺寸)
		self.参数+=预处理.参数
		self.rms+=预处理.rms
		输出_内=预处理.前向传播(输入)
        
        
		for i in range(self.层数):
			GRU层=GRU_单位_集束(self.隐藏层尺寸)
			self.参数+=GRU层.参数
			self.rms+=GRU层.rms
			输出_内=GRU层.前向传播(输出_内,输入_遮罩,集束量)
#		信息包=输出_内[-1]

		注意力层=GRU_注意力(self.隐藏层尺寸,self.隐藏层尺寸)
		self.参数+=注意力层.参数
		self.rms+=注意力层.rms            
		信息包=注意力层.前向传播(输出_内)            
        
		GRU层2=GRU_单位2_集束_A(self.隐藏层尺寸)
		self.参数+=GRU层2.参数
		self.rms+=GRU层2.rms
		输出_内2=GRU层2.前向传播(输出_遮罩,集束量,信息包)
#		信息包 = theano.printing.Print('this is 信息包 value')(信息包)        
		for i in range(self.层数-1):
			GRU层=GRU_单位_集束_A(self.隐藏层尺寸)
			self.参数+=GRU层.参数
			self.rms+=GRU层.rms
			输出_内2=GRU层.前向传播(输出_内2,输出_遮罩,集束量) 
            
            
		输出层=后处理_输出(self.词向量尺寸,self.隐藏层尺寸)
		self.参数+=输出层.参数
		self.rms+=输出层.rms
		输出_内2=输出层.前向传播(输出_内2)

		softmax_输出, updates = theano.scan(
			fn=lambda x: T.nnet.softmax(x.T),
			sequences=[输出_内2],
			)
# 		softmax_输出 = theano.printing.Print('this is softmax_输出 value')(softmax_输出)
		误差, updates = theano.scan(
			fn=lambda x,O:T.sum(T.nnet.categorical_crossentropy(x, O)),
			sequences=[softmax_输出,目标输出],
			)    
		误差 = T.sum(误差)




		参数集_导 = [T.grad(误差, 参数) for 参数 in self.参数]
		衰减系数 = T.scalar('L')
		学习率_变 = T.scalar('d')
		rms集= [衰减系数 * rms+ (1 - 衰减系数) * 参数_导 ** 2 for rms, 参数_导 in zip(self.rms, 参数集_导)]        
		参数_更新 = [(参数, 参数 - 学习率_变 * 参数_导/ T.sqrt(rms_更新 + 1e-6)) for 参数, 参数_导,rms_更新 in zip(self.参数, 参数集_导,rms集)]
		rms集_更新=[(rms, rms_更新) for rms,rms_更新 in zip(self.rms,rms集)]
		参数_更新+=rms集_更新               
        
		self.输出_预测 = theano.function(
					inputs=[输入,输入_遮罩,输出_遮罩,集束量],
					outputs=[softmax_输出],
					allow_input_downcast=True,
					on_unused_input='ignore'
					)
        
		self.训练 = theano.function(
			inputs=[输入,输入_遮罩,目标输出,输出_遮罩,集束量,theano.Param(学习率_变, default=0.1), theano.Param(衰减系数, default=0.9)],
			outputs=[误差],
			updates=参数_更新,
			allow_input_downcast=True,
					on_unused_input='ignore'
			)          
        
        
	def 预测(self, 输入,输入_遮罩,输出_遮罩,集束量):
		 return self.输出_预测(输入,输入_遮罩,输出_遮罩,集束量)
	def 训练(self, 输入,输入_遮罩,目标输出,输出_遮罩,集束量,学习率,衰减率):
		 return self.训练(输入,输入_遮罩,目标输出,输出_遮罩,集束量,学习率,衰减率)

In [13]:
np.random.seed(1)
import time
词汇总数 =13451
学习率 = 1e-3
隐藏层尺寸 = 512
层数=2

# print(包_输入[0].shape[1])
# #输出=模型2.预测误差(包_输入[0],包_输出[0],len(包_输出[0]),包_输入[0].shape[1])
# 输出=模型2.训练(包_输入[0],包_输出[0],len(包_输出[0]),包_输入[0].shape[1],学习率,0.9)
# print(输出[-1])

In [14]:
模型2 = GRU_集束(词汇总数 , 隐藏层尺寸,层数)

E:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:81: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)


In [ ]:
t1 = time.time()
for j in range(200):
    
    for i in range(70):

            输出=模型2.训练(包_输入[i],输入_遮罩_集[i],包_输出[i],输出_遮罩_集[i],包_输入[i].shape[1],学习率,0.9)
            if (i%10==0 and j%10==0):
                t2 = time.time()
                print("轮",j,"编号",i,输出[-1],'训练时间',(t2 - t1))

print ("单步训练时间: ~%f 秒" % ((t2 - t1) * 1.))
输出

E:\Anaconda\envs\py36\lib\site-packages\theano\tensor\subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


轮 0 编号 0 13144.098 训练时间 0.7055971622467041
轮 0 编号 10 11838.783 训练时间 8.117412328720093
轮 0 编号 20 12077.9375 训练时间 15.166855096817017
轮 0 编号 30 12181.998 训练时间 22.17899250984192
轮 0 编号 40 13072.996 训练时间 29.526125192642212
轮 0 编号 50 9527.252 训练时间 36.81300401687622
轮 0 编号 60 12165.01 训练时间 43.783461570739746
轮 10 编号 0 11328.76 训练时间 505.0467486381531
轮 10 编号 10 10460.002 训练时间 512.278555393219
轮 10 编号 20 10546.098 训练时间 519.478182554245
轮 10 编号 30 10751.404 训练时间 526.6720108985901
轮 10 编号 40 11966.244 训练时间 533.9835987091064
轮 10 编号 50 8081.5073 训练时间 541.1823704242706
轮 10 编号 60 10859.41 训练时间 548.3924095630646
轮 20 编号 0 10355.304 训练时间 1009.0517213344574
轮 20 编号 10 9235.482 训练时间 1016.0628020763397
轮 20 编号 20 9551.25 训练时间 1023.0388171672821
轮 20 编号 30 9538.997 训练时间 1030.1131699085236
轮 20 编号 40 10559.67 训练时间 1037.316630601883
轮 20 编号 50 6788.911 训练时间 1044.7008199691772
轮 20 编号 60 9644.483 训练时间 1052.0008516311646
轮 30 编号 0 9170.809 训练时间 1500.873844385147
轮 30 编号 10 8383.715 训练时间 1507.906108379364
轮 3

In [28]:
import sys
sys.setrecursionlimit(15000)
import pickle 


with open('data/模型2_6_26', "wb") as mf:
     pickle.dump(模型2, mf)


In [29]:
with open('data/模型2_6_26', "rb") as mf:
       模型3= pickle.load(mf)
    

In [44]:
输出初步[-1][:,-1,:].shape


13451

In [30]:
import sys
def print_sentence(s, index_to_word):
    sentence_str = [index_to_word[x] for x in s[1:-1]]
    print(" ".join(sentence_str))
    sys.stdout.flush()

def generate_sentence(model, index_to_word, word_to_index, min_length=5):
    # We start the sentence with the start token
    new_sentence = [word_to_index[SENTENCE_START_TOKEN]]
    print(new_sentence)
    # Repeat until we get an end token
    while not len(new_sentence)  < 20:
        next_word_probs = model.预测(new_sentence,1)[-1]
        samples = np.random.multinomial(1, next_word_probs[-1])
        sampled_word = np.argmax(samples)
        print(len(new_sentence))
        new_sentence.append(sampled_word)
        # Seomtimes we get stuck if the sentence becomes too long, e.g. "........" :(
        # And: We don't want sentences with UNKNOWN_TOKEN's
        if len(new_sentence) > 100 or sampled_word == word_to_index[UNKNOWN_TOKEN]:
            return None
    if len(new_sentence) < min_length:
        return None
    return new_sentence

def generate_sentences2(model, n, index_to_word, word_to_index):
    for i in range(n):
        sent = None
        while not sent:
            sent = generate_sentence(model, index_to_word, word_to_index)
        print_sentence(sent, index_to_word)

In [72]:
print(包_输入[0][:,1])
# oph=输入_遮罩_集[0][0,:]
# 测试1=1-oph[None,:]
# 测试2=np.ones((2,200))
# 测试3=测试1*测试2
# 测试3

[17872 17859 17827 12421 12971 17714 17814  3230 16627  3282   817 17501
 15443 17813 17870]


In [35]:
for j in range(51,52):
#     p2=p.ones_like(输入_遮罩_集[j])
    结果_1=模型3.预测(包_输入[j],输入_遮罩_集[j],输入_遮罩_集[j],包_输入[j].shape[1])

    
    for k in range(结果_1[-1].shape[1]):
        new_sentence = [单词_到_下标[SENTENCE_START_TOKEN]]
        for i in range(len(结果_1[-1][:,k,:])):
        #samples = np.random.multinomial(1, 结果[-1][i])
            

            sampled_word = np.argmax(结果_1[-1][i,k,:])

            new_sentence.append(sampled_word)

        if j>=0:

            #print("次数",j )
            print("输入" )
            print_sentence(包_输入[j][:,k], 下标_到_单词)
            #print("目标输出" )
            #print_sentence(包_输出[j][:,k], 下标_到_单词)
            #print_sentence(X_train[j], 下标_到_单词)
            #print(包_输入[j][:,k])
            
            print_sentence(new_sentence, 下标_到_单词)

输入
手机 都 要 50 万 也 太过分 了 稳如 手机 <MASK/> <MASK/> <MASK/>
谁 小 ， 不用 ？ 的 ？ 稳 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
精装 的 两万 也 就 买个 沙发 吧 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
最 的 布林德 有 嗨 了 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
+ 1 1 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
1 + 1 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
+ 1 + 1 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
+ 1 + 1 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
id 吓 尿 + 1 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
+ 1 我 楼主 SENTENCE_END 了 SENTENCE_END <MASK/> <MASK/> SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/>
输入
来 报道 + 1 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
好 还有 感觉 终于 请 一个 的 不 。 了 SENTENCE_END <MASK/> <MASK/> <MASK/>
输入
+ 7 + 1 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> 

？ ？ 了 不是 这 还有 还有 么 SENTENCE_END SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/>
输入
9 9 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
布林德 0 分 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
东方 教主 a <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
逆天 速度 可惜 不是 b 了 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
比起 小笼包 ， 还是 香肠 好 一些 都 是 垃圾 <MASK/> <MASK/> <MASK/>
好 了 是 感觉 爆 呢 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
a a <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
ck 来 的 来看 bilibili SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
必须 a a <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
萨 好久 我 好 SENTENCE_END SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
b 是 我 唯一 完整 看过 两遍 的 武侠剧 这个 ， 我选 

7 10 ， ， 很 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
9 我 又 一次 赞同 你 ！ <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
我 我 挺 终于 挺 好 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
9 7 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
7 8 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
100 分 6 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
6 0 SENTENCE_END SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
7 8 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
8 10 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
8 9 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
9 8 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/

输入
巴狗 死个 妈 先 演员 队 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
百度 直接 一个 一个 看 看 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
一日 白衣 ， 终身 傻 b ？ 看得出来 ， 真 红蓝 魂 <MASK/>
终于 这个 叫 就 一起 才 才 我 哈哈哈哈 ？ <MASK/> <MASK/> <MASK/> <MASK/>
输入
为什么 他们 不 叫 床单 ？ 就 你 话 多 <MASK/> <MASK/> <MASK/>
就 是 是 是 打 你 真 的 就是 吧 SENTENCE_END <MASK/> <MASK/> <MASK/>
输入
他 抬 我 大红 魔 ， 快快 崛起 吧 <MASK/> <MASK/> <MASK/> <MASK/>
有 有 我 都 有 有 有 ？ 有 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
我 魔 合影 抬 你 咋 地 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
有 有 有没有 我 可以 为什么 我 看过 我 … SENTENCE_END <MASK/> <MASK/> <MASK/>
输入
欧洲 亚军 杯 ， 拜仁 慕尼黑 五仁 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
鲁尼 ， ， 有 那 ， 有 有 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
移动 队 分期 队 <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
层主 挺 ， 的 了 确实 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/>
输入
队徽 上 也 有 两只 喵 就 你 话 多 <MASK/> <MASK/> <MASK/>
我 这么 🐶 吧 这 手机 退役 了 ！ <MASK/> <M

role A: 你 好 啊


In [34]:

def clearn_str(s):
	s = re.sub(r"\s+", " ", s)
	s = re.sub(r"\-", "", s)
	return 'SENTENCE_START '+s

def tokenize(s):
	"""
	very raw tokenizer
	"""
	#s = clearn_str(s)
	return s.strip().split(" ")
for ai in range(100):
    

    问话= input("role A: ")
    问话1=clearn_str(问话)

    问话_表=tokenize(问话1)

    问话_下标=[单词_到_下标[问话_分] for 问话_分 in  问话_表]
    长度=len(问话_下标)
    问话_下标=np.array(问话_下标)
   # print(问话_下标)
    问话_下标=问话_下标.reshape(长度,1)
    遮罩_入=np.ones_like(问话_下标)
    print(问话_下标.shape)
    print(遮罩_入.shape)
    for 计数1 in range(15-长度):
        问话_下标=np.append(问话_下标,0)
        遮罩_入=np.append(遮罩_入,0)
    问话_下标=问话_下标.reshape((15,1))
    遮罩_入=遮罩_入.reshape((15,1))
    遮罩_出=np.ones((15,1))
    结果_1=模型2.预测(问话_下标,遮罩_入,遮罩_出,问话_下标.shape[1])




    for k in range(结果_1[-1].shape[1]):
        new_sentence = [单词_到_下标[SENTENCE_START_TOKEN]]
        for i in range(len(结果_1[-1][:,k,:])):
            #samples = np.random.multinomial(1, 结果[-1][i])


            sampled_word = np.argmax(结果_1[-1][i,k,:])

            new_sentence.append(sampled_word)

        if j>=0:



            print_sentence(new_sentence, 下标_到_单词)
    #print_sentence(问话_下标, 下标_到_单词)

role A: 永远 活在 上赛季 ？ 永远 看 一个 赛季 
(9, 1)
(9, 1)
永远 看 一个 赛季 ？ 怎么 感觉 你 是 说 给 层主 听 的
role A: 永远 活在 上赛季 ？
(5, 1)
(5, 1)
永远 知道 你 怎么 ？ 怎么 怎么 说 ， 怎么 没 是 层主 的
role A: 漂亮 厉害 了 ， 我 的哥
(7, 1)
(7, 1)
厉害 了 ， 我 的哥 这 算是 最 成功 的 过 人 了 SENTENCE_END
role A: 漂亮 厉害 了 我 的哥
(6, 1)
(6, 1)
厉害 了 我 看 想 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> SENTENCE_END <MASK/>
role A: 漂亮 厉害 了 ， 我 的哥
(7, 1)
(7, 1)
厉害 了 ， 我 的哥 这 算是 最 成功 的 过 人 了 SENTENCE_END
role A: 漂亮 厉害 了 我 的哥
(6, 1)
(6, 1)
厉害 了 我 看 想 SENTENCE_END <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> <MASK/> SENTENCE_END <MASK/>
role A: 343434


KeyError: '343434'

[array([[[2.0278843e-08, 1.4345335e-08, 1.3121954e-08, ...,
          1.2518979e-08, 1.2646030e-08, 1.1545746e-08]],
 
        [[4.2733075e-09, 1.8196946e-12, 1.4060009e-12, ...,
          1.4425589e-12, 9.9022689e-13, 8.3998758e-13]],
 
        [[3.2288671e-02, 8.5213379e-08, 8.5515246e-08, ...,
          6.9670826e-08, 8.2567709e-08, 8.4449916e-08]],
 
        ...,
 
        [[9.9999106e-01, 2.2133615e-15, 2.2436844e-15, ...,
          1.7776041e-15, 2.1180311e-15, 2.0279282e-15]],
 
        [[9.9999750e-01, 6.7506461e-16, 6.6548155e-16, ...,
          5.6314970e-16, 6.9366429e-16, 6.0339520e-16]],
 
        [[9.9999940e-01, 6.3851821e-16, 6.4512618e-16, ...,
          5.2832531e-16, 6.9435129e-16, 5.7587690e-16]]], dtype=float32)]

In [57]:
包_输入[-1].shape

(15, 200)